In [1]:
import os
model_dir= os.path.join('/work/DLR','trained_models', 'MNIST', 'model4', 'nn_models/')

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import agop_gcnn as agc
from utils import trainer as tr
from torch.utils.data import Dataset
import random
import torch.backends.cudnn as cudnn
import rfm
import numpy as np
from trained_models.MNIST.model4 import trainer as t
import numpy as np
from sklearn.model_selection import train_test_split
from torch.linalg import norm
from torchvision import models
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from utils.groupy.gconv.pytorch_gconv.splitgconv2d import P4ConvZ2, P4ConvP4, P4MConvZ2, P4MConvP4M
from groupy.gconv.make_gconv_indices import *
from copy import deepcopy
from torch.nn.functional import pad
from torch.func import jacrev

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device='cpu'
print(f"Using device: {device}")


Using device: cuda:0


In [4]:
torch.cuda.empty_cache()

In [5]:
trainloader, valloader, testloader = t.get_loaders()
net= t.get_untrained_net()
init_net= deepcopy(net)
import os
if os.path.exists(model_dir+'mnist_gcnn_trained_nn.pth'):
    checkpoint = torch.load(model_dir+'mnist_gcnn_trained_nn.pth', map_location=torch.device(device), weights_only=True)
    net.load_state_dict(checkpoint['state_dict'])  # Access the 'state_dict' within the loaded dictionary
    print("Model weights loaded successfully.")

#print("Train the network first")
#t.train_net()

Model weights loaded successfully.


In [6]:
#tr.visualize_predictions(net, testloader, range(10), device, num_images=36)
print(tr.get_acc_ce(net.to(device), testloader))

/work/DLR/trained_models/MNIST/model4/model4.py:55: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


99.18


In [7]:
agc.verify_NFA(net, init_net, trainloader, layer_idx=0)

l_idx 0
Computing GOP for sample 0 out of 10
Computing GOP for sample 1 out of 10
Computing GOP for sample 2 out of 10
Computing GOP for sample 3 out of 10
Computing GOP for sample 4 out of 10
Computing GOP for sample 5 out of 10
Computing GOP for sample 6 out of 10
Computing GOP for sample 7 out of 10
Computing GOP for sample 8 out of 10
Computing GOP for sample 9 out of 10
Computing GOP for sample 10 out of 10
Shape after gradients:  torch.Size([9, 9])
Correlation between Trained Gcnn and AGOP:  tensor(0.9710, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Correlation between Un-trained Gcnn and AGOP:  tensor(0.8016, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)


tensor([[ 32.4009,  10.9018,  -0.0848,  14.6582,   9.7982,  -7.8775,  -0.9690,
          -7.6742, -13.6246],
        [ 10.9018,  40.6124,  15.8331,  -2.5075,  10.0508,  -0.9196, -10.2713,
         -11.8109,  -7.0028],
        [ -0.0848,  15.8331,  34.5695,  -8.2894,   9.9921,  11.9328, -16.0371,
         -10.1299,  -0.4323],
        [ 14.6582,  -2.5075,  -8.2894,  40.1040,  10.4525, -11.2240,  11.9465,
          -1.2527,  -8.9354],
        [  9.7982,  10.0508,   9.9921,  10.4525,  60.9114,  10.4478,   9.8888,
          10.5111,  10.0658],
        [ -7.8775,  -0.9196,  11.9328, -11.2240,  10.4478,  40.0207,  -8.5397,
          -3.1090,  14.2528],
        [ -0.9690, -10.2713, -16.0371,  11.9465,   9.8888,  -8.5397,  35.0250,
          16.5853,  -0.4319],
        [ -7.6742, -11.8109, -10.1299,  -1.2527,  10.5111,  -3.1090,  16.5853,
          41.6438,  11.1290],
        [-13.6246,  -7.0028,  -0.4323,  -8.9354,  10.0658,  14.2528,  -0.4319,
          11.1290,  32.3759]], dtype=torch.float6

In [10]:
agc.verify_NFA(net, init_net, trainloader, layer_idx=1)

l_idx 2
Computing GOP for sample 0 out of 10
Computing GOP for sample 1 out of 10
Computing GOP for sample 2 out of 10
Computing GOP for sample 3 out of 10
Computing GOP for sample 4 out of 10
Computing GOP for sample 5 out of 10
Computing GOP for sample 6 out of 10
Computing GOP for sample 7 out of 10
Computing GOP for sample 8 out of 10
Computing GOP for sample 9 out of 10
Computing GOP for sample 10 out of 10
Shape after gradients:  torch.Size([360, 360])
Correlation between Trained Gcnn and AGOP:  tensor(0.9770, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Correlation between Un-trained Gcnn and AGOP:  tensor(0.0836, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)


tensor([[ 1.5122,  0.5156,  0.1316,  ..., -0.3701,  0.1405, -0.4765],
        [ 0.5156,  1.7492,  0.1980,  ...,  0.2609,  0.0725, -0.2701],
        [ 0.1316,  0.1980,  1.8413,  ..., -0.6061, -0.3391, -0.5612],
        ...,
        [-0.3701,  0.2609, -0.6061,  ...,  3.3044,  0.7460, -0.0309],
        [ 0.1405,  0.0725, -0.3391,  ...,  0.7460,  2.1470,  0.8858],
        [-0.4765, -0.2701, -0.5612,  ..., -0.0309,  0.8858,  2.7916]],
       dtype=torch.float64)

In [11]:
agc.verify_NFA(net, init_net, trainloader, layer_idx=2)

l_idx 5
Computing GOP for sample 0 out of 10
Computing GOP for sample 1 out of 10
Computing GOP for sample 2 out of 10
Computing GOP for sample 3 out of 10
Computing GOP for sample 4 out of 10
Computing GOP for sample 5 out of 10
Computing GOP for sample 6 out of 10
Computing GOP for sample 7 out of 10
Computing GOP for sample 8 out of 10
Computing GOP for sample 9 out of 10
Computing GOP for sample 10 out of 10
Shape after gradients:  torch.Size([360, 360])
Correlation between Trained Gcnn and AGOP:  tensor(0.9334, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Correlation between Un-trained Gcnn and AGOP:  tensor(0.1745, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)


tensor([[ 1.4325,  0.1340, -0.2771,  ..., -0.0348, -0.1890,  0.1651],
        [ 0.1340,  1.2878, -0.2561,  ...,  0.2383, -0.0286,  0.0565],
        [-0.2771, -0.2561,  1.2542,  ..., -0.1439,  0.0051, -0.0365],
        ...,
        [-0.0348,  0.2383, -0.1439,  ...,  1.5052,  0.0078, -0.0170],
        [-0.1890, -0.0286,  0.0051,  ...,  0.0078,  1.3672,  0.2067],
        [ 0.1651,  0.0565, -0.0365,  ..., -0.0170,  0.2067,  1.4088]],
       dtype=torch.float64)

In [12]:
agc.verify_NFA(net, init_net, trainloader, layer_idx=3)

l_idx 7
Computing GOP for sample 0 out of 10
Computing GOP for sample 1 out of 10
Computing GOP for sample 2 out of 10
Computing GOP for sample 3 out of 10
Computing GOP for sample 4 out of 10
Computing GOP for sample 5 out of 10
Computing GOP for sample 6 out of 10
Computing GOP for sample 7 out of 10
Computing GOP for sample 8 out of 10
Computing GOP for sample 9 out of 10
Computing GOP for sample 10 out of 10
Shape after gradients:  torch.Size([720, 720])
Correlation between Trained Gcnn and AGOP:  tensor(0.9653, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Correlation between Un-trained Gcnn and AGOP:  tensor(0.0986, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)


tensor([[ 0.8657,  0.0083,  0.2049,  ..., -0.1020, -0.0330,  0.0882],
        [ 0.0083,  0.8676,  0.0457,  ..., -0.1038,  0.3065,  0.1029],
        [ 0.2049,  0.0457,  0.8468,  ...,  0.0284,  0.0456,  0.1190],
        ...,
        [-0.1020, -0.1038,  0.0284,  ...,  0.9005,  0.0070, -0.0297],
        [-0.0330,  0.3065,  0.0456,  ...,  0.0070,  0.7943, -0.0055],
        [ 0.0882,  0.1029,  0.1190,  ..., -0.0297, -0.0055,  0.7218]],
       dtype=torch.float64)

# Debug Task: Debug gcnn for AGOP

In [ ]:
def trans_filter(w, inds):
    inds_reshape = inds.reshape((-1, inds.shape[-1])).astype(np.int64)
    w_indexed = w[:, :, inds_reshape[:, 0].tolist(), inds_reshape[:, 1].tolist(), inds_reshape[:, 2].tolist()]
    w_indexed = w_indexed.view(w_indexed.size()[0], w_indexed.size()[1],
                                    inds.shape[0], inds.shape[1], inds.shape[2], inds.shape[3])
    w_transformed = w_indexed.permute(0, 2, 1, 3, 4, 5)
    return w_transformed.contiguous()
'''
layer_idx=2
count=0
# Get the layer_idx+1 th conv layer
for idx, m in enumerate(net.children()):
        if isinstance(m, P4ConvZ2):
            print("hi")
            count += 1
        if count-1 == layer_idx:
            l_idx = idx
      break
            '''
l_idx =7
layer = deepcopy(net.features[l_idx])
print(l_idx)
print(layer.weight)
print(layer.kernel_size)
print(layer.padding)
print(layer.stride)
# Extract W matrix
tw = trans_filter(layer.weight, layer.inds)
tw_shape = (layer.out_channels * layer.output_stabilizer_size,
                    layer.in_channels * layer.input_stabilizer_size,
                    layer.ksize, layer.ksize)
M = tw.view(tw_shape)


#_, ki, q, s = M.shape
#print(M.shape)
#_,ki, q, s = M.shape

k, ki, q,s= M.shape
            
# Build W which is a (k, c*q*s) matrix. What to do with ip_stab
M = M.reshape(-1, ki*q*s)
            
# Compute WtW which is (c*q*s,c*q*s) matrix
M = torch.einsum('nd, nD -> dD', M, M)


print(M)




In [ ]:
def correlation(M1, M2):
    M1 -= M1.mean()
    M2 -= M2.mean()

    norm1 = norm(M1.flatten())
    norm2 = norm(M2.flatten())

    return torch.sum(M1.cuda() * M2.cuda()) / (norm1 * norm2)

def patchify(x, in_channels, ip_stab, patch_size, stride_size, padding=None, pad_type='zeros'):
    '''
        Given an input image (n,c,h,w) generate (n,w_out,h_out,c,q,s) respecting stride,padding, 
        w_out is number of pathces along the width for the given stride after padding
        h_out is number of pathces along the height for the given stride after padding
        (q,s) is the kernel dimensions 
    '''
    input_shape = x.size()
    #TODO: The last two shapes look swapped. This was the same order in cohens code too. For square ips 
    #there is no effect. However for rect ips what would happen?
    x = x.view(input_shape[0], in_channels*ip_stab, input_shape[-2], input_shape[-1])
    #x = x.view(input_shape[0], in_channels*ip_stab, input_shape[-2], input_shape[-1])
    q1, q2 = patch_size
    s1, s2 = stride_size
    print("Image Shape",x.shape)
    if padding is None:
        pad_1 = (q1-1)//2
        pad_2 = (q2-1)//2
    else:
        pad_1, pad_2 = padding

    pad_dims = (pad_2, pad_2, pad_1, pad_1)
    if pad_type == 'zeros':
        x = pad(x, pad_dims)
    elif pad_type == 'circular':
        x = pad(x, pad_dims, 'circular')
        
    patches = x.unfold(2, q1, s1).unfold(3, q2, s2) #(n, c, h_out, w_out, q, s)
    print("Image Shape1",patches.shape)
    patches = patches.transpose(1, 3).transpose(1, 2) #(n,w_out,h_out,c,q,s) 
    print("Image Shape2",patches.shape)
    return patches

def trans_filter(w, inds):
    inds_reshape = inds.reshape((-1, inds.shape[-1])).astype(np.int64)
    w_indexed = w[:, :, inds_reshape[:, 0].tolist(), inds_reshape[:, 1].tolist(), inds_reshape[:, 2].tolist()]
    w_indexed = w_indexed.view(w_indexed.size()[0], w_indexed.size()[1],
                                    inds.shape[0], inds.shape[1], inds.shape[2], inds.shape[3])
    w_transformed = w_indexed.permute(0, 2, 1, 3, 4, 5)
    return w_transformed.contiguous()

class PatchConvLayer(nn.Module):
    def __init__(self, conv_layer):
        super().__init__()
        self.layer = conv_layer #(k,c,q,s)
        #inds = make_c4_z2_indices(self.layer.ksize)
       
    def forward(self, patches):
        tw = trans_filter(self.layer.weight, self.layer.inds)
        tw_shape = (self.layer.out_channels * self.layer.output_stabilizer_size,
                    self.layer.in_channels * self.layer.input_stabilizer_size,
                    self.layer.ksize, self.layer.ksize)
        tw = tw.view(tw_shape)
        print("tw shape",tw.shape)
        print("Patch_shape", patches.shape)
        #n, _, ny_out, nx_out = patches.shape()
        #print(tw)
        out = torch.einsum('nhwcqr, kcqr -> nhwk', patches, tw)
        n, w, h, k = out.shape
        out = out.transpose(1, 3).transpose(2, 3) #(n,k,h_out,w_out)
        out = out.view(n, self.layer.out_channels, self.layer.output_stabilizer_size, h, w)
        print("out_shape", out.shape)
        return out



for idx, batch in enumerate(trainloader):
    imgs,_=batch
    break

img=imgs[6:7]
l_idx=7
img=img.to(device)
img = img.double()
#layers= list(net.children())
#print("Shape of wight matrix:",layers[l_idx].weight.shape)
net=net.double()
net = net.to(device)
img = net.features[:l_idx](img)
print("Image shape",img.shape)



patchnet = deepcopy(net)
temp= deepcopy(net.features[l_idx])
layer = PatchConvLayer(temp)
patchnet.features = deepcopy(net.features[l_idx:])
patchnet.features[0] = layer
patchnet.to(device)
#op = patchnet(patchify(img, temp.kernel_size, temp.stride, temp.padding))
op1 = patchnet.features[0](patchify(img, temp.in_channels, 
                                    temp.input_stabilizer_size, temp.kernel_size, temp.stride, temp.padding))
print(op1.shape)
print(op1)
#print(patchnet2)


patchnet2 = deepcopy(net)
layer = deepcopy(net.features[l_idx])
patchnet2.features = deepcopy(net.features[l_idx:])
patchnet2.features[0] = layer
patchnet2.features[0].bias = None
patchnet2.features[0].padding = (0,0)
patchnet2.to(device)
#print(patchnet2)
#op2= patchnet2(img)
op2= patchnet2.features[0](img)
#print(op2.shape)
#print(op2)
print(torch.allclose(op1,op2,rtol=0.00000000001,atol=0, equal_nan=False))

def single_net(x):
            patchnet.eval()
            return patchnet(x).squeeze(0)

def single_net2(x):
            patchnet2.eval()
            return patchnet2(x).squeeze(0)

op=jacrev(single_net)(patchify(img, temp.in_channels, 
                                    temp.input_stabilizer_size, temp.kernel_size, temp.stride, temp.padding))
n, c, w, h, _, _, _ = op.shape
patches = op.transpose(1, 3).transpose(1, 2) #(n, h_out, w_out, chunk, c, q, s)
grads = patches.reshape(n*w*h, c, -1) #(n*w_out*h_out, chunk, c*q*s)
#print("Grads:",grads.shape)
#Clarify: Where is mean taken
ajop = torch.einsum('ncd, ncD -> dD', grads, grads) #(c*q*s,c*q*s)
ajop= sqrt(ajop)
print(correlation(ajop, M))


In [104]:
def co(M1):
    M= M1.clone()
    M -= M.mean()
    #M.flatten()
    return M
M= torch.randn(3, 3)
print(M)
co(M)
print(M)

tensor([[-0.1223, -1.2220,  1.0906],
        [ 0.2609, -1.7309, -1.0689],
        [-1.8210,  1.9365, -1.7288]])
tensor([[-0.1223, -1.2220,  1.0906],
        [ 0.2609, -1.7309, -1.0689],
        [-1.8210,  1.9365, -1.7288]])


# Debug Task: Check if correlation existst between WTW and AGOP with W not including rotations¶

In [28]:
''' This module does the following
1. Scan the network for conv layers
2. For each gcnn conv layer compute W^TW of eq 3
3. For each gcnn conv layer compute the AGOP(AJOP in case of multiple outputs)
4. For each gcnn conv layer print the pearson correlation between 2 and 3
'''

import torch
import torch.nn as nn
import random
import numpy as np
#from functorch import jacrev, vmap
from torch.func import jacrev
from torch.nn.functional import pad
#import dataset
#from numpy.linalg import eig
from copy import deepcopy
from torch.linalg import norm, svd
from torchvision import models
import visdom
from torch.linalg import norm, eig
#import torchvision
#import torchvision.transforms as transforms
import random
import torch.backends.cudnn as cudnn
from torch.linalg import norm
from torchvision import models
import torch.nn.functional as F
from utils.groupy.gconv.pytorch_gconv.splitgconv2d import P4ConvZ2, P4ConvP4, P4MConvZ2, P4MConvP4M
from groupy.gconv.make_gconv_indices import *
from copy import deepcopy
from torch.nn.functional import pad
from torch.func import jacrev

SEED = 2323

torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)

vis = visdom.Visdom('http://127.0.0.1', use_incoming_socket=False)
vis.close(env='main')

def patchify(x, in_channels, ip_stab, patch_size, stride_size, padding=None, pad_type='zeros'):
    '''
        Given an input image (n,c,h,w) generate (n,w_out,h_out,c,q,s) respecting stride,padding, 
        w_out is number of pathces along the width for the given stride after padding
        h_out is number of pathces along the height for the given stride after padding
        (q,s) is the kernel dimensions 
    '''
    input_shape = x.size()
    #TODO: The last two shapes look swapped. This was the same order in cohens code too. For square ips 
    #there is no effect. However for rect ips what would happen?
    x = x.view(input_shape[0], in_channels*ip_stab, input_shape[-2], input_shape[-1])
    #x = x.view(input_shape[0], in_channels*ip_stab, input_shape[-2], input_shape[-1])
    q1, q2 = patch_size
    s1, s2 = stride_size
    #print("Image Shape",x.shape)
    if padding is None:
        pad_1 = (q1-1)//2
        pad_2 = (q2-1)//2
    else:
        pad_1, pad_2 = padding

    pad_dims = (pad_2, pad_2, pad_1, pad_1)
    if pad_type == 'zeros':
        x = pad(x, pad_dims)
    elif pad_type == 'circular':
        x = pad(x, pad_dims, 'circular')
        
    patches = x.unfold(2, q1, s1).unfold(3, q2, s2) #(n, c, h_out, w_out, q, s)
    #print("Image Shape1",patches.shape)
    patches = patches.transpose(1, 3).transpose(1, 2) #(n,w_out,h_out,c,q,s) 
    #print("Image Shape2",patches.shape)
    return patches

def trans_filter(w, inds):
    inds_reshape = inds.reshape((-1, inds.shape[-1])).astype(np.int64)
    w_indexed = w[:, :, inds_reshape[:, 0].tolist(), inds_reshape[:, 1].tolist(), inds_reshape[:, 2].tolist()]
    w_indexed = w_indexed.view(w_indexed.size()[0], w_indexed.size()[1],
                                    inds.shape[0], inds.shape[1], inds.shape[2], inds.shape[3])
    w_transformed = w_indexed.permute(0, 2, 1, 3, 4, 5)
    return w_transformed.contiguous()
    
class PatchConvLayer(nn.Module):
    def __init__(self, conv_layer):
        super().__init__()
        self.layer = conv_layer #(k,c,q,s)
        #inds = make_c4_z2_indices(self.layer.ksize)
       
    def forward(self, patches):
        tw = trans_filter(self.layer.weight, self.layer.inds)
        tw_shape = (self.layer.out_channels * self.layer.output_stabilizer_size,
                    self.layer.in_channels * self.layer.input_stabilizer_size,
                    self.layer.ksize, self.layer.ksize)
        tw = tw.view(tw_shape)
        #print("tw shape",tw.shape)
        #print("Patch_shape", patches.shape)
        out = torch.einsum('nhwcqr, kcqr -> nhwk', patches, tw)
        n, w, h, k = out.shape
        out = out.transpose(1, 3).transpose(2, 3) #(n,k,h_out,w_out)
        out = out.view(n, self.layer.out_channels, self.layer.output_stabilizer_size, h, w)
        #print("out_shape", out.shape)
        return out

def get_jacobian(net, data, c_idx=0, chunk=100):
    with torch.no_grad():
        def single_net(x):
            # x is (w_out,h_out,c,q,s)
            return net(x.unsqueeze(0))[:,c_idx*chunk:(c_idx+1)*chunk].squeeze(0)
        # Parallelize across the images.
        return torch.vmap(jacrev(single_net))(data) #(n, chunk, w_out, h_out, c, q, s)

def egop(model, z):
    ajop = 0
    c = 10
    chunk_idxs = 1
    #Chunking is done to compute jacobian as chunks. This saves memory
    #TODO: chunk should be passed as argument
    chunk = c // chunk_idxs
    for i in range(chunk_idxs):
        J = get_jacobian(model, z, c_idx=i, chunk=chunk)
        n, c, w, h, _, _, _ = J.shape
        J = J.transpose(1, 3).transpose(1, 2) #(n, w_out, h_out, chunk, c, q, s)
        grads = J.reshape(n*w*h, c, -1) #(n*w_out*h_out, chunk, c*q*s)
        #Clarify: Where is mean taken
        ajop += torch.einsum('ncd, ncD -> dD', grads, grads) #(c*q*s,c*q*s)
    return ajop


def load_nn(net, init_net, layer_idx=0):
    
    count = 0
    # Get the layer_idx+1 th conv layer
    for idx, m in enumerate(net.features):
        if isinstance(m, (P4ConvZ2, P4ConvP4, P4MConvZ2, P4MConvP4M)):
            count += 1
        if count-1 == layer_idx:
            l_idx = idx
            break

    print("l_idx",l_idx)
    layer = deepcopy(net.features[l_idx])
    layer_init = deepcopy(init_net.features[l_idx])

    # Extract all the meta info of the current conv layer.
    (q, s) = net.features[l_idx].kernel_size
    (pad1, pad2) = net.features[l_idx].padding
    (s1, s2) = net.features[l_idx].stride 
    in_channels = layer.in_channels
    input_stabilizer_size = layer.input_stabilizer_size
    
    # Extract W matrix
    #tw = trans_filter(layer.weight, layer.inds)
    tw = layer.weight
    tw_shape = (layer.out_channels,
                        layer.in_channels * layer.input_stabilizer_size,
                        layer.ksize, layer.ksize)
    M = tw.view(tw_shape)
    
    #tw= trans_filter(layer_init.weight, layer_init.inds)
    tw = layer_init.weight
    tw_shape = (layer_init.out_channels,
                        layer_init.in_channels * layer_init.input_stabilizer_size,
                        layer_init.ksize, layer_init.ksize)
    M0 = tw.view(tw_shape)
    
    k, ki, q,s= M.shape
                
    # Build W which is a (k, c*q*s) matrix. What to do with ip_stab
    M = M.reshape(-1, ki*q*s)
                
    # Compute WtW which is (c*q*s,c*q*s) matrix
    M = torch.einsum('nd, nD -> dD', M, M)

    k, ki, q,s= M0.shape

    # Build W which is a (k, c*q*s) matrix. What to do with ip_stab
    M0 = M0.reshape(-1, ki*q*s)

    # Compute WtW which is (c*q*s,c*q*s) matrix
    M0 = torch.einsum('nd, nD -> dD', M0, M0)

    # Construct patchnet
    patchnet = deepcopy(net)
    temp = deepcopy(net.features[l_idx])
    layer = PatchConvLayer(temp)
    
    # Truncate all layers before l_idx    
    patchnet.features = net.features[l_idx:]
    patchnet.features[0] = layer

    return net, patchnet, M, M0, l_idx, [(q, s), (pad1,pad2), (s1,s2)], in_channels, input_stabilizer_size


def get_grads(net, in_channels, input_stabilizer_size, patchnet, trainloader,
              kernel=(3,3), padding=(1,1),
              stride=(1,1), layer_idx=0):
    net.eval()
    net.cuda()
    patchnet.eval()
    patchnet.cuda()
    M = 0
    q, s = kernel
    pad1, pad2 = padding
    s1, s2 = stride

    # Num images for taking AGOP (Can be small for early layers)
    MAX_NUM_IMGS = 10

    for idx, batch in enumerate(trainloader):
        print("Computing GOP for sample " + str(idx) + \
              " out of " + str(MAX_NUM_IMGS))
        imgs, _ = batch
        imgs= imgs.double()
        with torch.no_grad():
            imgs = imgs.cuda()        
            # Run the first half of the network wrt to the current layer 
            imgs = net.features[:layer_idx](imgs).cpu() #(n,c,h,w)
        patches = patchify(imgs, in_channels, input_stabilizer_size, 
                           (q, s), (s1,s2), padding=(pad1,pad2))#(n,w_out,h_out,c,q,s)
        patches = patches.cuda()
        #print(patches.shape)
        M += egop(patchnet, patches).cpu()
        del imgs, patches
        torch.cuda.empty_cache()
        if idx >= MAX_NUM_IMGS:
            break
    net.cpu()
    patchnet.cpu()
    return M


def min_max(M):
    return (M - M.min()) / (M.max() - M.min())


def correlation(A, B):
    M1 = A.clone()
    M2 = B.clone()
    M1 -= M1.mean()
    M2 -= M2.mean()

    norm1 = norm(M1.flatten())
    norm2 = norm(M2.flatten())

    return torch.sum(M1.cuda() * M2.cuda()) / (norm1 * norm2)


def verify_NFA(net, init_net, trainloader, layer_idx=0):

    net = net.double()
    init_net = init_net.double()
    net, patchnet, M, M0, l_idx, conv_vals, in_channels, input_stabilizer_size = load_nn(net,
                                                     init_net,
                                                     layer_idx=layer_idx)
    (q, s), (pad1, pad2), (s1, s2) = conv_vals
    '''
    i_val = correlation(M0, M)
    print("Correlation between Initial and Trained CNFM: ", i_val)'''

    G = get_grads(net, in_channels, input_stabilizer_size, patchnet, trainloader,
                  kernel=(q, s),
                  padding=(pad1, pad2),
                  stride=(s1, s2),
                  layer_idx=l_idx)
    print("Shape after gradients: ", G.shape)
    G = sqrt(G)
    Gop = G.clone()
    r_val = correlation(M, G)
    print("Correlation between Trained Gcnn and AGOP: ", r_val)
    
    i_val = correlation(M0, G)
    print("Correlation between Un-trained Gcnn and AGOP: ", i_val)

    #print("Final: ", i_val, r_val)
    return Gop 
    #return i_val.data.numpy(), r_val.data.numpy()



def sqrt(G):
    U, s, Vt = svd(G)
    s = torch.pow(s, 1./2)
    G = U @ torch.diag(s) @ Vt
    return G


#TODO: ADD a visualizer for the image

#if __name__ == "__main__":
    #main()

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


In [34]:
verify_NFA(net, init_net, trainloader, layer_idx=1)

l_idx 2
Computing GOP for sample 0 out of 10
Computing GOP for sample 1 out of 10
Computing GOP for sample 2 out of 10
Computing GOP for sample 3 out of 10
Computing GOP for sample 4 out of 10
Computing GOP for sample 5 out of 10
Computing GOP for sample 6 out of 10
Computing GOP for sample 7 out of 10
Computing GOP for sample 8 out of 10
Computing GOP for sample 9 out of 10
Computing GOP for sample 10 out of 10
Shape after gradients:  torch.Size([360, 360])
Correlation between Trained Gcnn and AGOP:  tensor(0.5332, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Correlation between Un-trained Gcnn and AGOP:  tensor(0.0432, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)


tensor([[ 1.5122,  0.5156,  0.1316,  ..., -0.3701,  0.1405, -0.4765],
        [ 0.5156,  1.7492,  0.1980,  ...,  0.2609,  0.0725, -0.2701],
        [ 0.1316,  0.1980,  1.8413,  ..., -0.6061, -0.3391, -0.5612],
        ...,
        [-0.3701,  0.2609, -0.6061,  ...,  3.3044,  0.7460, -0.0309],
        [ 0.1405,  0.0725, -0.3391,  ...,  0.7460,  2.1470,  0.8858],
        [-0.4765, -0.2701, -0.5612,  ..., -0.0309,  0.8858,  2.7916]],
       dtype=torch.float64)

In [36]:
verify_NFA(net, init_net, trainloader, layer_idx=2)

l_idx 5
Computing GOP for sample 0 out of 10
Computing GOP for sample 1 out of 10
Computing GOP for sample 2 out of 10
Computing GOP for sample 3 out of 10
Computing GOP for sample 4 out of 10
Computing GOP for sample 5 out of 10
Computing GOP for sample 6 out of 10
Computing GOP for sample 7 out of 10
Computing GOP for sample 8 out of 10
Computing GOP for sample 9 out of 10
Computing GOP for sample 10 out of 10
Shape after gradients:  torch.Size([360, 360])
Correlation between Trained Gcnn and AGOP:  tensor(0.5206, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Correlation between Un-trained Gcnn and AGOP:  tensor(0.0932, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)


tensor([[ 1.4325,  0.1340, -0.2771,  ..., -0.0348, -0.1890,  0.1651],
        [ 0.1340,  1.2878, -0.2561,  ...,  0.2383, -0.0286,  0.0565],
        [-0.2771, -0.2561,  1.2542,  ..., -0.1439,  0.0051, -0.0365],
        ...,
        [-0.0348,  0.2383, -0.1439,  ...,  1.5052,  0.0078, -0.0170],
        [-0.1890, -0.0286,  0.0051,  ...,  0.0078,  1.3672,  0.2067],
        [ 0.1651,  0.0565, -0.0365,  ..., -0.0170,  0.2067,  1.4088]],
       dtype=torch.float64)

In [46]:
print(net.features)

Sequential(
  (0): P4ConvZ2()
  (1): ReLU()
  (2): P4ConvP4()
  (3): ReLU()
  (4): PlaneGroupSpatialMaxPool()
  (5): P4ConvP4()
  (6): ReLU()
  (7): P4ConvP4()
  (8): ReLU()
  (9): PlaneGroupSpatialMaxPool()
)


In [48]:
print(net.classifier)

Sequential(
  (0): Linear(in_features=1280, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=10, bias=True)
)


In [60]:
import torch
import torch.nn as nn
import torchvision.models as models

# --- 1. Output for Image Classification (Default Behavior) ---

# Load a pre-trained ResNet-18 model
# 'pretrained=True' downloads the weights trained on ImageNet
resnet18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
print(resnet18)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  